## 1.0 DATA CLEANING and EDA for amazon-fine-food-reviews dataset

In [1]:
import sqlite3
import pandas as pd

# Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews
con = sqlite3.connect("amazon-fine-food-reviews/database.sqlite")

# filtering only positive and negative reviews using SQL queries, removing scores having score = 3
# as they are neutral reviews
filtered_data = pd.read_sql_query(""" SELECT * from REVIEWS where Score != 3""", con)



In [6]:
filtered_data.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


In [3]:
# review having score 1 and 2 are negative
# review having score 4 and 5 are positive
def review_intent(x):
    if x<3:
        return 'negative'
    else:
        return 'positive'
    
Score = filtered_data['Score']

review_category = Score.map(review_intent)
filtered_data['Score'] = review_category


In [7]:
filtered_data.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


In [5]:
filtered_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

## 1.1 Data Cleaning: Deduplication

* It is observed that many reviews are duplicates which are posted exactly at the same time, that is the case because 
manufacturer is adding the same review to similar products. By similar product means the products which have very little difference like color etc.

In [17]:
display=pd.read_sql_query("""
SELECT *
from Reviews
where Score != 3 and UserId = 'AR5J8UI46CURR'
ORDER BY ProductId""",con)

display.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


* Time, ProfileName , UserId are same. When we go to amazon site see these products, we found these products are exactly the same products,
and have small variations like color, flavour etc .

In [8]:
# sorting data according to ProductId in ascending order
sorted_filter_data = filtered_data.sort_values('ProductId', axis = 0 , ascending= True)

print(sorted_filter_data.shape)


(525814, 10)


In [18]:
# removing the rows which duplicates depending on 4 feature 

final_data = sorted_filter_data.drop_duplicates(subset={"UserId",'ProfileName','Text','Time'}, keep='first',inplace=False)

print(final_data.shape)



(364173, 10)


In [19]:
# percentage elements which are duplicates

print("Total elements which are duplicates are {}".format(100 - 364173/525814*100))

Total elements which are duplicates are 30.741098563370315


## 1.2 Observation

* HelpfulnessNumerator : People said YES that review is useful.
* HelpfulnessDenominator : People said YES or NO that review is useful.
* So HelpfulnessNumerator will always be less than HelpfulnessDenominator.



* removing the rows which are not following above observation, there might be some mistake. So two rows were faulty.


In [21]:
print(final_data.shape)
final_data = final_data[final_data.HelpfulnessNumerator <= final_data.HelpfulnessDenominator]
print(final_data.shape)

(364173, 10)
(364171, 10)


In [28]:
final_data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."


In [25]:
## counting positive and negative reviews
print("Total filtered reviews are : {}".format(final_data.shape[0]))
print("Total positive reviews are : {}".format(final_data[final_data.Score == 'positive'].shape[0]))
print("Total negative reviews are : {}".format(final_data[final_data.Score == 'negative'].shape[0]))

Total filtered reviews are : 364171
Total positive reviews are : 307061
Total negative reviews are : 57110


In [27]:
# second way to print positive and negative counts
final_data['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

## 1.3 Text Preprocessing : Stemming, Stop-Word removal, Tokenization, Lemmitization

In [31]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('english'))

print(stop_words)

{'ourselves', "weren't", "shouldn't", 'very', "won't", 'in', "isn't", 'am', 'during', 'above', 'ma', "that'll", 'of', 'but', "you'd", 'himself', 'itself', "you've", 'ain', 'between', 'down', 'own', 'this', 'both', 'your', "needn't", 'she', 'few', 's', "haven't", 'each', 'through', 'won', "should've", 'into', 'then', 'than', "don't", 've', 'out', "you'll", 'having', 'her', 'o', 'further', 'did', 'other', 'had', "doesn't", 'mustn', 'yourselves', "wouldn't", 'whom', 'been', 'should', 'a', 'shan', 'being', "you're", 'they', 'below', 'i', 'myself', 'which', 'that', 'now', 'its', 'more', 'for', "wasn't", 'against', 'up', 'these', 'doing', 'have', "mustn't", 'him', 'those', 'does', 'again', 'were', 'what', 'wouldn', 'their', 'from', 'isn', 'about', 'needn', 'his', 'as', 'me', "she's", 'once', 'when', 'any', 'too', "didn't", 'if', 'don', 'is', 'until', 'how', 'all', 'doesn', 'weren', 'wasn', 'm', 'and', 'hasn', 'most', 'y', 'yours', 'at', 'haven', 'do', 'theirs', 'with', 'nor', 'couldn', 'by',

* In above we are seeing lot of words having negative sentiment as stop words, we can not remove them,
because by removing them the intent of the reviews will be totally reversed.

In [ ]:
    sno_stemmer = nltk.stem.SnowballStemmer('english')
    